In [24]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle

In [13]:
df = pd.read_csv("coffee_data.csv")

In [14]:
df.head()

,temperature,time,target
0,185.317638,12.693965,1.0
1,259.920475,11.867664,0.0
2,231.013571,14.414242,0.0
3,175.366645,11.720587,0.0
4,187.120865,14.129732,1.0


In [15]:
X = df.loc[:,df.columns != "target"]
y = df.target

print(X.shape, y.shape)

(200, 2) (200,)


In [16]:
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
Xn = norm_l(X)

In [17]:
Xt = np.tile(Xn,(1000,1))
Yt= np.tile(y,(1000))   
print(Xt.shape, Yt.shape)

(200000, 2) (200000,)


In [18]:
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(3, activation='sigmoid', name = 'layer1'),
        Dense(1, activation='sigmoid', name = 'layer2')
     ]
)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [21]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.08995402  0.8367007  -0.8995967 ]
 [ 0.13073432 -0.22296411  0.6252128 ]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[-0.3891139 ]
 [ 0.07862532]
 [-0.65564656]] 
b2(1,): [0.]


In [22]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1886
Epoch 2/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1178
Epoch 3/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0790
Epoch 4/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0194
Epoch 5/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0119
Epoch 6/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0082
Epoch 7/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0058
Epoch 8/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0042
Epoch 9/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0030
Epoch 10/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0022


In [23]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[ 14.482052    -0.04834193 -10.765552  ]
 [ 12.057639    -9.034746    -0.1905933 ]] 
b1: [  1.7829647 -11.264209  -11.741996 ]
W2:
 [[-41.29246 ]
 [-44.52932 ]
 [-48.338894]] 
b2: [25.420885]


In [26]:
X_input = np.array([200,13.9]).reshape(1,-1)
input = norm_l(X_input)
prediction = model.predict(input)
yhat = (prediction >= 0.5).astype(int)

1/1 [==============================] - 0s 75ms/step


In [27]:
print(prediction)
print(yhat)

[[0.987678]]
[[1]]


In [25]:
pickle.dump(model, open("coffee_model.pkl", "wb"))

INFO:tensorflow:Assets written to: ram://0295a3de-f84e-4922-8cac-ab33f5658f0a/assets
